# Based on the date from 3. we can see taht characteristics have the best indication for analyst forecast, now we will compare and condition on the other criteria to se if this holds true. We will be removing covid years, as this seems to give better results.

In [15]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects

Data

In [16]:
df = pd.read_csv("Dataframes/characteristics_regression.csv")
#X_var_names = ["Revenue - Actual","Enterprise Value","Market Capitalization","loss firm status", "Enterprise Value To Sales (Daily Time Series Ratio)", "3 Month Total Return", "Volume"]
df["Date"] = pd.to_datetime(df["Date"])
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,11.178,1.532200e+09,3.426001e+09,0.799347,10.637479,207295.0,-1
1,AVY.N,2013-04-01,Materials,2.482,2.482,1.498900e+09,4.309428e+09,0.895787,24.148726,0.0,-1
2,AVY.N,2013-07-01,Materials,1.068,1.068,1.552300e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
3,AVY.N,2013-10-01,Materials,8.095,8.095,1.504900e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
4,AVY.N,2014-01-01,Materials,1.471,1.471,1.583900e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,17.194,1.411448e+09,1.742020e+10,3.684400,-5.130322,157740.0,-1
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,1.035557e+09,2.268979e+10,4.504382,30.489705,82349.0,-1
19198,POOL.OQ,2022-04-01,Retailing,34.342,34.342,1.412650e+09,1.696712e+10,3.422880,-25.160664,290897.0,-1
19199,POOL.OQ,2022-07-01,Retailing,1.503,1.503,2.055818e+09,1.405795e+10,2.749481,-21.661570,185584.0,-1


Standardising

In [17]:
rescale = df.copy()
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))
rescale["3 Month Total Return"] = MinMaxScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Volume"] = MinMaxScaler().fit_transform(np.array(rescale["Volume"]).reshape(-1,1))
df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,11.178,0.065123,0.001166,0.027195,0.298034,0.001183,-1
1,AVY.N,2013-04-01,Materials,2.482,2.482,0.064917,0.001471,0.027308,0.340793,0.000000,-1
2,AVY.N,2013-07-01,Materials,1.068,1.068,0.065247,0.001453,0.027321,0.264198,0.000000,-1
3,AVY.N,2013-10-01,Materials,8.095,8.095,0.064955,0.001459,0.027317,0.272179,0.001348,-1
4,AVY.N,2014-01-01,Materials,1.471,1.471,0.065443,0.001667,0.027338,0.315017,0.000900,-1
...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,17.194,0.064377,0.006004,0.030598,0.248134,0.000900,-1
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.062055,0.007826,0.031566,0.360861,0.000470,-1
19198,POOL.OQ,2022-04-01,Retailing,34.342,34.342,0.064385,0.005848,0.030290,0.184744,0.001660,-1
19199,POOL.OQ,2022-07-01,Retailing,1.503,1.503,0.068358,0.004842,0.029495,0.195817,0.001059,-1


### Create 2 new dataframes one with only covid years one with only non-covid years

In [18]:
covid_start = pd.to_datetime("2020-01-01")

df_covid = df_clean[df_clean["Date"] >= covid_start]

df_nocovid = df_clean[df_clean["Date"] < covid_start]

#### Fixed Effects

In [19]:
df_nocovid['Instrument'] = df_nocovid['Instrument'].astype('category')
df_nocovid['Date'] = pd.to_datetime(df_nocovid['Date'])

# set the index to be the time variable and the cross-sectional variable
df_nocovid.set_index(['Instrument', 'Date'], inplace=True)

C:\Users\alexa\AppData\Local\Temp\ipykernel_16524\876088522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nocovid['Instrument'] = df_nocovid['Instrument'].astype('category')
C:\Users\alexa\AppData\Local\Temp\ipykernel_16524\876088522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nocovid['Date'] = pd.to_datetime(df_nocovid['Date'])


#### FE Panel Regression with Absolute Suprise Values for Covid and non-Covid quarters

Non-Covid Quarters

In [20]:
y = df_nocovid.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_nocovid.loc[:, "Revenue - Actual":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\linearmodels\panel\model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0048
Estimator:,PanelOLS,R-squared (Between):,-0.4960
No. Observations:,13222,R-squared (Within):,0.0057
Date:,"Sun, Mar 05 2023",R-squared (Overall):,-0.2097
Time:,20:49:54,Log-likelihood,-6.277e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,10.287
Entities:,498,P-value,0.0000
Avg Obs:,26.550,Distribution:,"F(6,12692)"
Min Obs:,0.0000,,
Max Obs:,28.000,F-statistic (robust):,10.287


# Conditioning on Analyst dataframe

In [20]:
analysts = pd.read_csv("Dataframes/analysts_regression.csv")

In [ ]:
#analysts["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(analysts["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
analysts["Earnings Per Share – Coefficient of Variation"] = MinMaxScaler().fit_transform(np.array(analysts["Earnings Per Share – Coefficient of Variation"]).reshape(-1,1))
analysts["Number of Analysts"] = MinMaxScaler().fit_transform(np.array(analysts["Number of Analysts"]).reshape(-1,1))
analysts["Recommendation - Mean (1-5)"] = MinMaxScaler().fit_transform(np.array(analysts["Recommendation - Mean (1-5)"]).reshape(-1,1))
analysts["Recommendation change"] = MinMaxScaler().fit_transform(np.array(analysts["Recommendation change"]).reshape(-1,1))
analysts["Recommendation - Mean (1-5).1"] = MinMaxScaler().fit_transform(np.array(analysts["Recommendation - Mean (1-5).1"]).reshape(-1,1))
analysts["Price Target - Standard Deviation"] = MinMaxScaler().fit_transform(np.array(analysts["Price Target - Standard Deviation"]).reshape(-1,1))


# Conditioning on Macro Dataframe

In [21]:
macro = pd.read_csv("Dataframes/macro_regression.csv")

In [ ]:
#macro["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(macro["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
macro["CBOE Crude Oil ETF Volatility Index"] = MinMaxScaler().fit_transform(np.array(macro["CBOE Crude Oil ETF Volatility Index"]).reshape(-1,1))
macro["90-Day AA Financial Commercial Paper Interest Rate"] = MinMaxScaler().fit_transform(np.array(macro["90-Day AA Financial Commercial Paper Interest Rate"]).reshape(-1,1))
macro["Inflation Risk Premium"] = MinMaxScaler().fit_transform(np.array(macro["Inflation Risk Premium"]).reshape(-1,1))
macro["University of Michigan: Consumer Sentiment, Index 1966:Q1=100"] = MinMaxScaler().fit_transform(np.array(macro["University of Michigan: Consumer Sentiment, Index 1966:Q1=100"]).reshape(-1,1))
macro["Unemployment Rate"] = MinMaxScaler().fit_transform(np.array(macro["Unemployment Rate"]).reshape(-1,1))
